# Scryfall API Call to Dataframe
### API Doc https://scryfall.com/docs/api

### Small Sample <<< TEST HERE

In [1]:
#import requests
#import pandas as pd

# Define your search query
#query = "c:red pow=3"
#encoded_query = requests.utils.quote(query)

# Construct the full URL
#url = f"https://api.scryfall.com/cards/search?order=cmc&q={encoded_query}"

# Make a single request (no loops)
#response = requests.get(url)
#response.raise_for_status()  # raise an error for bad responses

# Extract JSON data and normalize it into a DataFrame
#data = response.json()
#cards = data["data"]
#df = pd.json_normalize(cards)

# Preview key fields
#print(df[["name", "colors", "power", "toughness", "set_name"]].head())


In [2]:
#df

In [3]:
# Save to xlsx
# df.to_csv("magic_scry_unprocessed.csv", index=False, encoding="utf-8")

### Full Bulk <<< START HERE

In [4]:
import requests
import pandas as pd

# Step 1: Get metadata about available bulk data
bulk_response = requests.get("https://api.scryfall.com/bulk-data")
bulk_response.raise_for_status()
bulk_data = bulk_response.json()

# Step 2: Find the 'default_cards' file (entire card database)
default_cards_info = next(
    item for item in bulk_data["data"] if item["type"] == "default_cards"
)

download_url = default_cards_info["download_uri"]

# Step 3: Download the full card data
cards_response = requests.get(download_url)
cards_response.raise_for_status()
cards_json = cards_response.json()

# Step 4: Convert to DataFrame
df = pd.json_normalize(cards_json)
print(f"Retrieved {len(df)} cards")

Retrieved 107796 cards


### Add additional columns is_multicolor + is_colorless based on color_identity list length

In [5]:
## Create is_multicolor
df["is_multicolor"] = df["color_identity"].apply(
    lambda x: isinstance(x, list) and len(x) > 1
)

## Create is_colorless
df["is_colorless"] = df["color_identity"].apply(
    lambda x: isinstance(x, list) and len(x) == 0
)

# Functions

## Expand List

In [7]:
def expand_nested_column(df, id_col, nested_col):
    """
    Expands a column containing lists of dicts into separate columns.

    Parameters:
        df (pd.DataFrame): The original DataFrame.
        id_col (str): The name of the column to keep (e.g., 'id').
        nested_col (str): The name of the column with nested data (e.g., 'legalities').

    Returns:
        pd.DataFrame: A DataFrame with the id and expanded nested fields.
    """
    def parse_nested(nested_list):
        if isinstance(nested_list, list):
            return {item.get("format") or item.get("date"): item.get("legality") or item.get("text")
                    for item in nested_list if isinstance(item, dict)}
        return {}

    parsed_series = df[nested_col].apply(parse_nested)
    expanded_df = pd.json_normalize(parsed_series)
    result_df = pd.concat([df[[id_col]].reset_index(drop=True), expanded_df.reset_index(drop=True)], axis=1)
    return result_df

## Explode List

In [8]:
def explode_list_column(df, id_col, list_col):
    """
    Explodes a list column into long format, preserving the ID.

    Parameters:
        df (pd.DataFrame): Source DataFrame.
        id_col (str): ID column name.
        list_col (str): Column containing list of strings.

    Returns:
        pd.DataFrame: A long-format DataFrame with one row per list item.
    """
    df_copy = df[[id_col, list_col]].copy()
    df_copy = df_copy.explode(list_col).dropna().reset_index(drop=True)
    df_copy.columns = [id_col, list_col.rstrip('s')]  # Optional: singular column name
    return df_copy

## Explode Dictionary

In [9]:
def explode_dict_column(df, id_col, list_col):
    """
    Explodes a list-of-dicts column into a normalized DataFrame.

    Parameters:
    df (pd.DataFrame): Source DataFrame
    id_col (str): Name of the ID column (to retain as foreign key)
    list_col (str): Name of the column that contains list of dicts

    Returns:
    pd.DataFrame: A new DataFrame where each dict is a row, with 'id' retained
    """
    # Step 1: Keep only ID and target column
    subset_df = df[[id_col, list_col]].copy()

    # Step 2: Filter out rows where target column is not a list
    subset_df = subset_df[subset_df[list_col].apply(lambda x: isinstance(x, list))]

    # Step 3: Explode the list column
    exploded_df = subset_df.explode(list_col)

    # Step 4: Normalize dictionaries into separate columns
    normalized_df = pd.json_normalize(exploded_df[list_col])

    # Step 5: Add ID column back
    normalized_df[id_col] = exploded_df[id_col].values

    return normalized_df

## DIM Color (One-To-Many)

In [10]:
# Explode the 'keywords' column
colors_dim_df = explode_list_column(df, id_col="id", list_col="color_identity")

In [11]:
colors_dim_df

,id,color_identity
0,0000419b-0bba-4488-8f7a-6194544ce91e,G
1,0000579f-7b35-4ed3-b44c-db2a538066fe,R
2,00006596-1166-4a79-8443-ca9f82e6db4e,W
3,0000a54c-a511-4925-92dc-01b937f9afad,W
4,0000cd57-91fe-411f-b798-646e965eec37,U
...,...,...
116494,fffc85fb-1a40-4f83-a36e-cec0b7be658a,U
116495,fffce2f7-b619-4483-a75e-916343194641,G
116496,fffdf7f3-a230-417a-883a-069aabcbcca7,B
116497,fffe7b2b-22c3-4e6a-9b1b-c6d7b29b9f86,W


## Dim Keyword Ability (One-To-Many)
### Solution: Explode List

In [12]:
# Explode the 'keywords' column
keywords_dim_df = explode_list_column(df, id_col="id", list_col="keywords")

In [13]:
keywords_dim_df

,id,keyword
0,0000a54c-a511-4925-92dc-01b937f9afad,Flying
1,0000cd57-91fe-411f-b798-646e965eec37,Flying
2,0000cd57-91fe-411f-b798-646e965eec37,Explore
3,00012bd8-ed68-4978-a22d-f450c8a6e048,Enchant
4,0001c639-8bd0-426f-89cb-4ca61f3cc054,Paradox
...,...,...
60323,fff9989f-77a3-4f73-ade6-c04306c98501,Morbid
60324,fff9ed67-3c45-48ff-a1e7-f95ff35b782b,Cycling
60325,fffc85fb-1a40-4f83-a36e-cec0b7be658a,Fight
60326,fffdf7f3-a230-417a-883a-069aabcbcca7,Flying


## DIM Rarity (One-to-One)

In [14]:
ordered_rarities = ['common', 'uncommon', 'rare', 'mythic', 'bonus', 'special']
unique_rarities = df['rarity'].dropna().unique()

# Warn if unexpected values appear
unexpected = set(unique_rarities) - set(ordered_rarities)
if unexpected:
    print(f"⚠️ Unexpected rarities detected: {unexpected}")

# Use controlled order
rarity_mapping = {rarity: idx + 1 for idx, rarity in enumerate(ordered_rarities)}

In [15]:
# Add rarity_id to main df
df['rarity_id'] = df['rarity'].map(rarity_mapping)

In [16]:
# Create dim_rarity_df
dim_rarity_df = pd.DataFrame({
    'rarity': ordered_rarities 
})
dim_rarity_df['rarity_id'] = dim_rarity_df.index + 1

In [17]:
dim_rarity_df

,rarity,rarity_id
0,common,1
1,uncommon,2
2,rare,3
3,mythic,4
4,bonus,5
5,special,6


## DIM Set (One-to-Many) perspective of a set

In [18]:
# Step 1: Get unique set combinations
dim_set_df = df[['set', 'set_name', 'set_type']].drop_duplicates().reset_index(drop=True)

# Step 2: Create a unique set_id if needed (optional)
dim_set_df['set_id'] = dim_set_df.index + 1

# Step 3: Reorder columns
dim_set_df = dim_set_df[['set_id', 'set', 'set_name', 'set_type']]


In [19]:
# Step 4: Create a mapping from set code to set_id
set_mapping = dict(zip(dim_set_df['set'], dim_set_df['set_id']))

# Step 5: Map it into fact_card table
df['set_id'] = df['set'].map(set_mapping)

In [20]:
dim_set_df

,set_id,set,set_name,set_type
0,1,blb,Bloomburrow,expansion
1,2,tsp,Time Spiral,expansion
2,3,zen,Zendikar,expansion
3,4,tmm2,Modern Masters 2015 Tokens,token
4,5,xln,Ixalan,expansion
...,...,...,...,...
968,969,pchk,Champions of Kamigawa Promos,promo
969,970,ovoc,Crimson Vow Commander Display Commanders,memorabilia
970,971,ppcy,Prophecy Promos,promo
971,972,plny,Lunar New Year 2018,promo


## DIM Legalities (One-to-One)

In [21]:
dim_legalities_df=df[['id','legalities.standard','legalities.future','legalities.timeless','legalities.gladiator','legalities.pioneer','legalities.modern','legalities.legacy','legalities.pauper','legalities.vintage','legalities.penny','legalities.commander','legalities.oathbreaker','legalities.standardbrawl','legalities.brawl','legalities.alchemy','legalities.paupercommander','legalities.duel','legalities.oldschool','legalities.premodern','legalities.predh']]

In [22]:
dim_legalities_df.columns = dim_legalities_df.columns.str.replace('legalities.', '', regex=False)

In [23]:
dim_legalities_df

,id,standard,future,timeless,gladiator,pioneer,modern,legacy,pauper,vintage,...,commander,oathbreaker,standardbrawl,brawl,alchemy,paupercommander,duel,oldschool,premodern,predh
0,0000419b-0bba-4488-8f7a-6194544ce91e,legal,legal,legal,legal,legal,legal,legal,legal,legal,...,legal,legal,legal,legal,legal,legal,legal,not_legal,legal,legal
1,0000579f-7b35-4ed3-b44c-db2a538066fe,not_legal,not_legal,not_legal,not_legal,not_legal,legal,legal,not_legal,legal,...,legal,legal,not_legal,not_legal,not_legal,not_legal,legal,not_legal,not_legal,legal
2,00006596-1166-4a79-8443-ca9f82e6db4e,not_legal,not_legal,not_legal,not_legal,not_legal,legal,legal,legal,legal,...,legal,legal,not_legal,not_legal,not_legal,legal,legal,not_legal,not_legal,legal
3,0000a54c-a511-4925-92dc-01b937f9afad,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,...,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal
4,0000cd57-91fe-411f-b798-646e965eec37,not_legal,not_legal,legal,legal,legal,legal,legal,legal,legal,...,legal,legal,not_legal,legal,not_legal,legal,legal,not_legal,not_legal,not_legal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107791,fffcbc4e-c6dc-4808-b262-f7c453e74dd8,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,...,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal
107792,fffce2f7-b619-4483-a75e-916343194641,not_legal,not_legal,not_legal,not_legal,not_legal,legal,legal,legal,legal,...,legal,legal,not_legal,not_legal,not_legal,legal,legal,not_legal,legal,legal
107793,fffdf7f3-a230-417a-883a-069aabcbcca7,not_legal,not_legal,not_legal,not_legal,not_legal,not_legal,legal,not_legal,legal,...,legal,legal,not_legal,not_legal,not_legal,not_legal,legal,not_legal,not_legal,not_legal
107794,fffe7b2b-22c3-4e6a-9b1b-c6d7b29b9f86,not_legal,not_legal,legal,legal,legal,legal,legal,legal,legal,...,legal,legal,not_legal,legal,not_legal,legal,legal,not_legal,not_legal,not_legal


## DIM Type (One-to-One)

In [24]:
df['type_line'].head()

0        Basic Land — Forest
1          Creature — Sliver
2     Creature — Kor Soldier
3    Token Creature — Spirit
4    Creature — Siren Pirate
Name: type_line, dtype: object

In [25]:
# Regex with tighter matching logic
df[['supertypes', 'types', 'subtypes']] = df['type_line'].str.extract(
    r'^(?:(?P<supertypes>[\w ]+?) )?(?P<types>[^—]+?)(?: — (?P<subtypes>.*))?$'
)

# Clean whitespace
df['supertypes'] = df['supertypes'].str.strip()
df['types'] = df['types'].str.strip()
df['subtypes'] = df['subtypes'].str.strip()

# Replace empty strings or all whitespace with NA
for col in ['supertypes', 'types', 'subtypes']:
    df[col].replace(r'^\s*$', pd.NA, regex=True, inplace=True)

#create dim
dim_type_df = df[['id', 'supertypes', 'types', 'subtypes']]

In [26]:
# Get unique combinations of the 3 columns
dim_type_df = (
    df[['supertypes', 'types', 'subtypes']]
    .drop_duplicates()
    .reset_index(drop=True)
    .copy()
)

# Add a surrogate key
dim_type_df.insert(0, 'type_id', range(1, len(dim_type_df) + 1))

In [27]:
# Merge to bring in the surrogate key
df = df.merge(dim_type_df, on=['supertypes', 'types', 'subtypes'], how='left')

In [28]:
dim_type_df[['type_id','supertypes','types','subtypes']]

,type_id,supertypes,types,subtypes
0,1,Basic,Land,Forest
1,2,NaN,Creature,Sliver
2,3,NaN,Creature,Kor Soldier
3,4,Token,Creature,Spirit
4,5,NaN,Creature,Siren Pirate
...,...,...,...,...
4360,4361,NaN,Creature,Human Scout Werewolf // Creature — Werewolf
4361,4362,Token,Creature,Spirit Warrior
4362,4363,NaN,Creature,Capybara
4363,4364,NaN,Creature,Ninja


## Price Fact Table (With Daily Insert)

In [39]:
from datetime import datetime
import os

# Prepare today's price data
price_fact_df_new = df[['id', 'prices.usd', 'prices.usd_foil', 'prices.usd_etched',
                        'prices.eur', 'prices.eur_foil', 'prices.tix']].copy()

price_fact_df_new['date_loaded'] = pd.to_datetime('today').normalize()

# Master file path
master_path = "price_fact_df_master.csv"

if os.path.exists(master_path):
    # Read existing master and get max ID
    price_fact_df_master = pd.read_csv(master_path)
    starting_id = price_fact_df_master['price_fact_id'].max() + 1

    # Assign new IDs to today's data
    price_fact_df_new.insert(0, 'price_fact_id', range(starting_id, starting_id + len(price_fact_df_new)))

    # Append and save
    updated_df = pd.concat([price_fact_df_master, price_fact_df_new], ignore_index=True)
    updated_df.to_csv(master_path, index=False)
else:
    # First run: start IDs from 1
    price_fact_df_new.insert(0, 'price_fact_id', range(1, len(price_fact_df_new) + 1))
    price_fact_df_new.to_csv(master_path, index=False)


In [40]:
# Create snapshot backup
today_str = datetime.today().strftime('%Y-%m-%d')
price_fact_df_new.to_csv(f"price_fact_snapshot_{today_str}.csv", index=False)

In [41]:
price_fact_df_new

,price_fact_id,id,prices.usd,prices.usd_foil,prices.usd_etched,prices.eur,prices.eur_foil,prices.tix,date_loaded
0,538940,0000419b-0bba-4488-8f7a-6194544ce91e,0.24,0.51,None,0.28,0.41,0.03,2025-07-03
1,538941,0000579f-7b35-4ed3-b44c-db2a538066fe,0.32,3.79,None,0.25,1.28,0.03,2025-07-03
2,538942,00006596-1166-4a79-8443-ca9f82e6db4e,0.13,1.70,None,0.29,1.88,0.03,2025-07-03
3,538943,0000a54c-a511-4925-92dc-01b937f9afad,0.10,None,None,None,None,None,2025-07-03
4,538944,0000cd57-91fe-411f-b798-646e965eec37,0.03,0.29,None,0.02,0.17,0.03,2025-07-03
...,...,...,...,...,...,...,...,...,...
107791,646731,fffcbc4e-c6dc-4808-b262-f7c453e74dd8,2.22,None,None,0.43,None,None,2025-07-03
107792,646732,fffce2f7-b619-4483-a75e-916343194641,0.09,None,None,0.16,None,0.03,2025-07-03
107793,646733,fffdf7f3-a230-417a-883a-069aabcbcca7,1.46,None,None,0.79,None,2.10,2025-07-03
107794,646734,fffe7b2b-22c3-4e6a-9b1b-c6d7b29b9f86,0.06,0.68,None,0.04,0.21,0.03,2025-07-03


## Card Fact Table

In [42]:
fact_card_df=df[['id','released_at','name','mana_cost','cmc','power','toughness','is_multicolor','is_colorless','rarity_id','set_id','type_id']]

In [43]:
fact_card_df

,id,released_at,name,mana_cost,cmc,power,toughness,is_multicolor,is_colorless,rarity_id,set_id,type_id
0,0000419b-0bba-4488-8f7a-6194544ce91e,2024-08-02,Forest,,0.0,NaN,NaN,False,False,1,1,1
1,0000579f-7b35-4ed3-b44c-db2a538066fe,2006-10-06,Fury Sliver,{5}{R},6.0,3,3,False,False,2,2,2
2,00006596-1166-4a79-8443-ca9f82e6db4e,2009-10-02,Kor Outfitter,{W}{W},2.0,2,2,False,False,1,3,3
3,0000a54c-a511-4925-92dc-01b937f9afad,2015-05-22,Spirit,,0.0,1,1,False,False,1,4,4
4,0000cd57-91fe-411f-b798-646e965eec37,2017-09-29,Siren Lookout,{2}{U},3.0,1,2,False,False,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
107791,fffcbc4e-c6dc-4808-b262-f7c453e74dd8,2009-09-30,Celestine Reef,,0.0,NaN,NaN,False,True,3,312,477
107792,fffce2f7-b619-4483-a75e-916343194641,2003-07-28,Horned Troll,{2}{G},3.0,2,2,False,False,1,139,476
107793,fffdf7f3-a230-417a-883a-069aabcbcca7,2023-09-08,Faerie Bladecrafter,{2}{B},3.0,2,2,False,False,3,36,594
107794,fffe7b2b-22c3-4e6a-9b1b-c6d7b29b9f86,2018-01-19,Exultant Skymarcher,{1}{W}{W},3.0,2,3,False,False,1,218,200


## Filtering on Card Name 'The Wise Mothman'

In [ ]:
#df[df["name"] == 'The Wise Mothman'][['id','released_at','name','mana_cost','cmc','power','toughness','is_multicolor','is_colorless']]

In [ ]:
#df[df["name"] == 'The Wise Mothman'][['id','prices.usd','prices.usd_foil','prices.usd_etched','prices.eur','prices.eur_foil','prices.tix']]

In [ ]:
#df[df["name"] == 'The Wise Mothman'][['id','artist','foil','promo_types','finishes','oversized','variation','reprint']]

In [ ]:
#df[df["name"] == 'The Wise Mothman'][['artist','image_uris.normal']]

In [ ]:
# Set max column width to None (no limit)
# pd.set_option('display.max_colwidth', None)